<a href="https://colab.research.google.com/github/pedrorfig/house-search/blob/main/pesquisa_imoveis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests, pandas as pd
from google.colab import files
from bs4 import BeautifulSoup
import json

In [2]:
bairros_sp = [ "Pinheiros", "Chácara Santo Antônio", "Vila Olímpia", "Itaim Bibi"]
# bairros_rj = ["Flamengo","Copacabana", "Botafogo", "Humaitá", "Jardim Botânico", "Catete","Bairro Peixoto", "Leme",  "Laranjeiras", "Leblon", "Ipanema", "Lagoa"]

In [3]:
def make_clickable(val):
    return '<a href="{}">{}</a>'.format(val, val)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
def get(cidade = "São Paulo", pais ="", estado = "São Paulo", zona = "", bairros =[], rua="", numero="", \
        tipo="venda",imovel ="apartamentos", areamin = 80,areamax = 200, paginas = 10, ordem_m2 = False, termos = ""):
  ll = []
  root = "https://www.zapimoveis.com.br"
  path = (root,tipo,imovel)
  pd.set_option('display.max_colwidth', None)

  for bairro in bairros: 
    localizacao = (pais,estado,cidade, zona, bairro, rua, numero)
    for pag in range(1,paginas):
      payload={
          "onde":",".join(localizacao),
          "areaMinima":areamin,
          "areaMaxima":areamax,
          "termos":termos,
          "pagina":pag}
      res = requests.get("/".join(path),params=payload,headers={'User-Agent': 'Mozilla/5.0'})
      soup = BeautifulSoup(res.text,'html.parser')
      card = soup.select(".js-listing-card")
      for i in range(0,len(card)):
          l = []
          try:
            preco = int(card[i].select(".js-price")[0].get_text().replace("/mês", "").replace("R$", "").replace(".", "").strip())
          except:
            continue
          try:
            cond = int(card[i].select(".condominium ")[0].get_text().replace("condomínio", "").replace("R$", "").replace(".", "").strip())
          except:
            continue
          try:
            iptu = int(card[i].select(".iptu")[0].get_text().replace("IPTU", "").replace("R$", "").replace(".", "").strip())
            if iptu > 500:
              iptu = round(iptu/12)
          except:
            iptu = 0
          try:
            area= int(card[i].select(".js-areas")[0].get_text().replace(" m²", "").replace(".", "").strip())
          except:
            area = 0
          try:
            quartos = int(card[i].select(".js-bedrooms")[0].get_text().strip())
          except:
            quartos = 0
          try:
            address = card[i].select(".simple-card__address")[0].get_text().strip()
          except:
            address = "-"
          if tipo == "venda":
            custo_total = round(preco)
          else:
            custo_total = round(preco+cond+iptu)
          try:
            custo_m2 = round(custo_total/area)
          except:
            custo_m2 = 0
          url = make_clickable(res.url)
          l = [address, bairro, preco, cond, iptu, area,quartos, custo_total, custo_m2, url]
          ll.append(l)
  df = pd.DataFrame(data=ll,columns = ["endereço", "bairro", "preço", "condominio","iptu", "area","quartos", "custo total", "custo m²", "url"])
  if ordem_m2:
    df.sort_values(by="custo m²", inplace = True)

  return df.style

In [6]:
get(tipo="aluguel",bairros=bairros_sp,paginas=10, ordem_m2=True)

,endereço,bairro,preço,condominio,iptu,area,quartos,custo total,custo m²,url
152,"Rua Teodoro Sampaio, Pinheiros",Pinheiros,2000,395,265,123,3,2660,22,"https://www.zapimoveis.com.br/aluguel/apartamentos/?areaMaxima=200&areaMinima=80&onde=,S%C3%A3o%20Paulo,S%C3%A3o%20Paulo,,Pinheiros,,&pagina=7&termos="
153,"Rua Teodoro Sampaio, Pinheiros",Pinheiros,2000,395,255,110,3,2650,24,"https://www.zapimoveis.com.br/aluguel/apartamentos/?areaMaxima=200&areaMinima=80&onde=,S%C3%A3o%20Paulo,S%C3%A3o%20Paulo,,Pinheiros,,&pagina=7&termos="
186,"Rua Teodoro Sampaio, Pinheiros",Pinheiros,2000,395,255,110,3,2650,24,"https://www.zapimoveis.com.br/aluguel/apartamentos/?areaMaxima=200&areaMinima=80&onde=,S%C3%A3o%20Paulo,S%C3%A3o%20Paulo,,Pinheiros,,&pagina=8&termos="
67,"Rua Cardeal Arcoverde, Pinheiros",Pinheiros,2500,300,50,95,2,2850,30,"https://www.zapimoveis.com.br/aluguel/apartamentos/?areaMaxima=200&areaMinima=80&onde=,S%C3%A3o%20Paulo,S%C3%A3o%20Paulo,,Pinheiros,,&pagina=3&termos="
192,"Rua Cardeal Arcoverde, Pinheiros",Pinheiros,2500,300,50,95,2,2850,30,"https://www.zapimoveis.com.br/aluguel/apartamentos/?areaMaxima=200&areaMinima=80&onde=,S%C3%A3o%20Paulo,S%C3%A3o%20Paulo,,Pinheiros,,&pagina=9&termos="
92,"Pinheiros, São Paulo",Pinheiros,2500,300,50,95,2,2850,30,"https://www.zapimoveis.com.br/aluguel/apartamentos/?areaMaxima=200&areaMinima=80&onde=,S%C3%A3o%20Paulo,S%C3%A3o%20Paulo,,Pinheiros,,&pagina=4&termos="
55,"Rua dos Pinheiros, Pinheiros",Pinheiros,2500,610,290,110,2,3400,31,"https://www.zapimoveis.com.br/aluguel/apartamentos/?areaMaxima=200&areaMinima=80&onde=,S%C3%A3o%20Paulo,S%C3%A3o%20Paulo,,Pinheiros,,&pagina=3&termos="
23,"Rua dos Pinheiros, Pinheiros",Pinheiros,2500,610,290,110,2,3400,31,"https://www.zapimoveis.com.br/aluguel/apartamentos/?areaMaxima=200&areaMinima=80&onde=,S%C3%A3o%20Paulo,S%C3%A3o%20Paulo,,Pinheiros,,&pagina=1&termos="
556,"Rua Jesuíno Arruda, Itaim Bibi",Itaim Bibi,2900,823,89,110,3,3812,35,"https://www.zapimoveis.com.br/aluguel/apartamentos/?areaMaxima=200&areaMinima=80&onde=,S%C3%A3o%20Paulo,S%C3%A3o%20Paulo,,Itaim%20Bibi,,&pagina=6&termos="
145,"Rua Capitão Prudente, Pinheiros",Pinheiros,2700,625,238,100,2,3563,36,"https://www.zapimoveis.com.br/aluguel/apartamentos/?areaMaxima=200&areaMinima=80&onde=,S%C3%A3o%20Paulo,S%C3%A3o%20Paulo,,Pinheiros,,&pagina=7&termos="


In [7]:
df = get(tipo="aluguel",bairros=bairros_sp,paginas=3, ordem_m2=True)

In [8]:
df.to_excel(r"/content/drive/MyDrive/Pesquisa_Imoveis/resultado.xlsx")

files.download('/content/drive/MyDrive/Pesquisa_Imoveis/resultado.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>